## 실습 문제: N-Puzzle 문제 정의

**설명**:
인공지능 탐색 알고리즘을 적용하기 위해 N-Puzzle(슬라이딩 퍼즐) 문제를 `Problem`이라는 추상 기본 클래스(Abstract Base Class)를 상속받아 정의합니다. N-Puzzle은 $N \times N$ 격자에서 빈 칸(0)을 움직여 숫자들을 오름차순으로 정렬하는 게임입니다. 상태(State)는 튜플로 표현되며, 행동(Action)은 빈 칸의 이동 방향('UP', 'DOWN', 'LEFT', 'RIGHT')으로 정의됩니다. 주어진 코드 뼈대에서 상태 전이 로직을 구현하세요.

$$s_{next} = Result(s_{current}, a)$$

**요구사항**:

1.  `actions` 메서드: 현재 상태에서 빈 칸(0)의 위치를 찾고, 격자 경계를 벗어나지 않는 유효한 이동 방향('UP', 'DOWN', 'LEFT', 'RIGHT')들의 리스트를 반환하도록 구현하세요.
2.  `result` 메서드: 현재 상태와 행동을 입력받아, 빈 칸과 해당 방향의 숫자를 서로 교환(Swap)한 \*\*새로운 상태(튜플)\*\*를 반환하도록 구현하세요. (기존 상태를 변경하면 안 됩니다.)
3.  `initial_state`인 `(7, 2, 4, 5, 0, 6, 8, 3, 1)`에서 시작하여, 가능한 행동들을 출력하고 'UP' 행동을 취했을 때의 결과 상태를 출력하여 검증하세요.


In [ ]:
import time
from abc import ABC, abstractmethod
from collections import deque
from typing import override

# --- 1. 문제 정의 인터페이스 ---
# S: State (상태), A: Action (행동)
class Problem[S, A](ABC):
    """
    모든 탐색 문제의 공통 조상 클래스입니다.
    """
    @abstractmethod
    def initial_state(self) -> S:
        pass

    @abstractmethod
    def actions(self, state: S) -> list[A]:
        pass

    @abstractmethod
    def result(self, state: S, action: A) -> S:
        pass

    @abstractmethod
    def goal_test(self, state: S) -> bool:
        pass

    @abstractmethod
    def step_cost(self, state: S, action: A) -> float:
        pass

In [5]:
# --- 2. N-Puzzle 문제 구체화 ---
class NPuzzleProblem(Problem[tuple[int, ...], str]):
    """
    N-Puzzle 문제를 정의하는 클래스입니다.
    """
    def __init__(self, n: int = 3, initial_state: tuple[int, ...] | None = None):
        self.n = n
        # 목표 상태 생성: (1, 2, ..., 8, 0)
        goal_tiles = list(range(1, n * n))
        goal_tiles.append(0)
        self.goal = tuple(goal_tiles)

        if initial_state:
            self._initial_state = initial_state
        else:
            # 기본 3x3 예제 상태 (Hard Case)
            if n == 3:
                self._initial_state = (7, 2, 4, 5, 0, 6, 8, 3, 1)
            else:
                self._initial_state = self.goal

    @override
    def initial_state(self) -> tuple[int, ...]:
        return self._initial_state

    @override
    def goal_test(self, state: tuple[int, ...]) -> bool:
        return state == self.goal

    @override
    def actions(self, state: tuple[int, ...]) -> list[str]:
        possible_actions: list[str] = []
        # 퍼즐의 빈 칸(숫자 0)의 위치
        blank_index = state.index(0)

        row, col = divmod(blank_index, self.n)

        # Implement your code
        if row != 0 : possible_actions.append('UP')
        if row != self.n-1 : possible_actions.append('DOWN')
        if col != 0 : possible_actions.append('LEFT')
        if col != self.n-1 : possible_actions.append('RIGHT')
        return possible_actions
        pass

    @override
    def result(self, state: tuple[int, ...], action: str) -> tuple[int, ...]:
        # 퍼즐의 빈 칸(숫자 0)의 위치
        blank_index = state.index(0)
        target_index = blank_index

        # Implement your code
        res = list(state)
        if action == 'UP' : target_index -= self.n
        if action == 'DOWN' : target_index += self.n
        if action == 'LEFT' : target_index -= 1
        if action == 'RIGHT' : target_index += 1
        res[target_index], res[blank_index] = res[blank_index], res[target_index]
        return tuple(res)
        pass

    @override
    def step_cost(self, state: tuple[int, ...], action: str) -> float:
        return 1.0

## 실습 문제: N-Puzzle 해결을 위한 BFS Solver 구현

**설명**:
앞서 정의한 N-Puzzle 문제(`Problem` 클래스)를 해결하기 위해 **너비 우선 탐색(Breadth-First Search, BFS)** 알고리즘을 구현합니다. BFS는 시작 상태에서 가까운 노드부터 차례대로 탐색하므로, 가중치가 없는 그래프(또는 모든 간선의 비용이 1인 경우)에서 **최단 경로**를 보장합니다. 탐색 효율을 높이기 위해 자식 노드를 생성하자마자 목표 상태인지 확인하는

$$O(b^d)$$
*(여기서 $b$는 분기 계수(branching factor), $d$는 해의 깊이(depth)를 의미합니다.)*

**요구사항**:

  - `deque`를 사용하여 `frontier` (탐색 큐)를 초기화하고, 중복 탐색 방지를 위한 `visited` 집합을 관리하세요.
  - 시작 상태가 목표인지 먼저 확인(로또!)하고, 이후 반복문 내에서 자식 노드를 생성할 때마다 목표 상태인지 즉시 확인하여 탐색 속도를 높이세요.
  - 목표 상태에 도달하면 `_reconstruct_path(child_node)`를 호출하여 탐색된 경로(행동 리스트)를 반환하세요.
  - 탐색에 실패하면 `None, []`을 반환하세요.

In [8]:
# --- 3. 탐색 노드 및 Solver ---
class Node[S, A]:
    """
    탐색 트리의 노드입니다.
    """
    def __init__(self, state: S, parent: 'Node[S, A] | None', action: A | None):
        self.state = state
        self.parent = parent
        self.action = action

class Solver[S, A]:
    def __init__(self, problem: Problem[S, A]):
        self.problem = problem
        self.visited_count = 0

    def _reconstruct_path(self, node: Node[S, A]) -> list[A]:
        """목표 노드에서 시작 노드까지 거슬러 올라가 경로 복원"""
        moves: list[A] = []
        current = node
        while current.parent is not None:
            if current.action is not None:
                moves.append(current.action)
            current = current.parent
        moves.reverse()
        return moves

    def breadth_first_search(self) -> tuple[S | None, list[A]]:
        """
        BFS 구현 (collections.deque 대신 기본 list 사용)
        """
        self.visited_count = 0
        initial_state = self.problem.initial_state()

        if self.problem.goal_test(initial_state):
            return initial_state, []

        initial_node = Node[S, A](
            state=initial_state,
            parent=None,
            action=None
        )

        frontier: list[Node[S, A]] = [initial_node]
        visited: set[S] = {initial_state}

        while frontier:
            current_node = frontier.pop(0)
            self.visited_count += 1

            # Implement your code
            if self.problem.goal_test(current_node.state):
                return current_node.state, self._reconstruct_path(current_node)
            
            for action in self.problem.actions(current_node.state) :
                next_state = self.problem.result(current_node.state, action)
                if next_state in visited : continue
                visited.add(next_state)
                frontier.append(Node(next_state, current_node, action))

        return None, []

In [11]:
# --- 4. 실행 ---
def print_board(state: tuple[int, ...], n: int) -> None:
    print(f"   {'-' * (n * 3)}")
    for i in range(n * n):
        tile = state[i]
        if i % n == 0: print("   |", end="")

        if tile == 0: print(" _ ", end="")
        else: print(f"{tile:2} ", end="")

        if (i + 1) % n == 0: print("|")
    print(f"   {'-' * (n * 3)}")

if __name__ == "__main__":
    hard_state = (7, 2, 4, 5, 0, 6, 8, 3, 1)

    problem = NPuzzleProblem(n=3, initial_state=hard_state)
    solver = Solver(problem)

    print(f"🧩 N-Puzzle Solver (Python 3.12+ Style)")
    print(f"---------------------------------------")
    print_board(problem.initial_state(), problem.n)

    print(f"\n🚀 BFS 탐색 시작...")
    start_time = time.time()

    solution_state, moves = solver.breadth_first_search()

    elapsed_time = time.time() - start_time

    if solution_state:
        print(f"\n✅ 성공!")
        print(f"   ⏱️ 시간: {elapsed_time:.4f}s")
        print(f"   🔍 노드: {solver.visited_count:,}")
        print(f"   👣 이동: {len(moves)}회")
        print(f"   📍 경로: {moves}")
        print_board(solution_state, problem.n)
    else:
        print(f"\n❌ 실패.")

🧩 N-Puzzle Solver (Python 3.12+ Style)
---------------------------------------
   ---------
   | 7  2  4 |
   | 5  _  6 |
   | 8  3  1 |
   ---------

🚀 BFS 탐색 시작...

✅ 성공!
   ⏱️ 시간: 0.1442s
   🔍 노드: 55,409
   👣 이동: 20회
   📍 경로: ['DOWN', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'DOWN', 'LEFT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']
   ---------
   | 1  2  3 |
   | 4  5  6 |
   | 7  8  _ |
   ---------
